# MNIST Problem

El MNIST problem es un problema de reconocimiento de imagenes que consiste en la clasificacion de digitos del 0 al 1 a partir de un dataset que contiene una gran cantidad de digitos escritos a mano.

Estos digitos estan contenidos en imagenes de 28x28 pixeles (784 pixeles) que pueden tomar valores de 0 a 255 que representan su intensidad, siendo 0 igual a negro y 255 blanco. Lo que va a hacer la deep network es transformar esta matriz de pixeles a un vector de entrada de dimensiones 784x1, y la misma tendra tantas entradas como pixeles.

La red tendra una capa de entrada, con 784 nodos, dos hidden layers y una capa de salida con 10 nodos (una por cada categoria), con una funcion de activacion Softmax. Se usara one-hot encoding para la representacion de las salidas y los targets.

## Importando Librerias

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Obteniendo el Dataset

In [3]:
# as_supervised lo que hace es descargar el dataset en una estructura de inputs y targets para aprendizaje superisado
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

## Preprocessing

In [6]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# cantidad de muestras destinadas al validation set
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # convierte el resultado a int64

# cantidad de muestras que posee el test set
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    # lo que hace esta funcion es escalar los valores de los pixeles de una imagen a valores entre 0 y 1, al dividir
    # por 255, ya que los valores de los pixeles (intensidad) van de 0 a 255
    
    image = tf.cast(image, tf.float64) # convierte los valores de los pixeles a float32 primero
    image /= 255.
    return image, label

# se aplica el escalado a todas las imagenes del train set
scaled_train_data = mnist_train.map(scale)

# se aplica el escalado a todas las imagenes del test set
test_data = mnist_test.map(scale)


# cantidad de muestras que se van a agarrar por vez para realizar el mezcaldo de los datos evitando que se tenga
# que poner todo el dataset en memoria de una sola vez, sobre todo para datasets muy grandes
BUFFER_SIZE = 10000

# se realiza un mezclado de las muestras del data set por las dudas vinieran ordenados segun algun parametro
shuffled_train_data = scaled_train_data.shuffle(BUFFER_SIZE)

# se toman las primeras num_validation_samples muestras para el validation set, y las siguientes para el train set
validation_data = shuffled_train_data.take(num_validation_samples)
train_data = shuffled_train_data.skip(num_validation_samples)

# batch size para el gradient descent
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

# para el validation set y el test set, no hace falta batching, ya que no se aplica el gradient descent en estos casos,
# pero el modelo espera el dataset el forma de batch tambien, pero eso definimos un unico batch con todas las muestras
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# validation_inputs, validation_targets = next(iter(validation_data))

## Inicializando el Modelo

In [10]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    # en la capa de entrada se aplica un faltten a las entradas, que consiste en tranformar la matriz de pixeles
    # de las imagenes de entrada (28, 28, 1) en un vector de dimensiones (784, 1), ya que esto no es una red convolucional
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [11]:
# sparse_categorical_crossentropy es un tipo de categorical crossentropy que aplica one-hot encoding, ya que el formato
# de las outputs y de los targets deben coincidir y estar en one-hot encoding, cosa que no hemos hecho

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Entrenando el Modelo

In [12]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data, verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.3324 - accuracy: 0.9069 - val_loss: 0.1896 - val_accuracy: 0.9457
Epoch 2/5
540/540 - 10s - loss: 0.1400 - accuracy: 0.9593 - val_loss: 0.1245 - val_accuracy: 0.9627
Epoch 3/5
540/540 - 10s - loss: 0.1009 - accuracy: 0.9697 - val_loss: 0.1024 - val_accuracy: 0.9702
Epoch 4/5
540/540 - 10s - loss: 0.0754 - accuracy: 0.9772 - val_loss: 0.0809 - val_accuracy: 0.9765
Epoch 5/5
540/540 - 10s - loss: 0.0623 - accuracy: 0.9811 - val_loss: 0.0709 - val_accuracy: 0.9802


## Testeando el Modelo

In [16]:
test_loss, test_accuracy = model.evaluate(test_data)

# print('Test loss: {0: .2f}. - Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100.))

      1/Unknown - 1s 1s/step - loss: 0.0911 - accuracy: 0.9723